In [ ]:

!pip install langchain langchain-openai langchain-community langchain-text-splitters chromadb sentence-transformers -q
print("LangChain + Embeddings Installed")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.3 MB/s eta 0:00:0

In [ ]:
from google.colab import userdata
import os, numpy as np
NEBIUS_KEY = userdata.get("NEBIUS_API_KEY") or os.getenv("NEBIUS_API_KEY")
if not NEBIUS_KEY:
    raise RuntimeError("Put NEBIUS_API_KEY into Colab Runtime Variables")
NEBIUS_BASE = "https://api.studio.nebius.com/v1"



In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI( model="meta-llama/Meta-Llama-3.1-8B-Instruct", base_url="https://api.studio.nebius.com/v1", api_key=NEBIUS_KEY, temperature=0.0, )
print("Nebius Chat Model Ready")

Nebius Chat Model Ready


In [ ]:
!pip install rank_bm25


In [ ]:


from langchain_community.retrievers import BM25Retriever


# --- Sample corpus ---
texts = [
    "Library rules: penalty fee for late books is $10 per week.",
    "Attendance rule: students must attend 75% of classes to be eligible for exams.",
    "How to apply for class leave: submit a form to the office.",
    "Library timings: Mon-Fri 9AM to 7PM, Sat 10AM to 4PM.",
    "Academic calendar: semester dates, holidays, and exam schedules."
]

# --- Create BM25 retriever from raw texts ---
bm25 = BM25Retriever.from_texts(texts)
bm25.k = 3  # number of top docs to retrieve

# --- Query ---
query = "library penalty fee rules"

results = bm25.get_relevant_documents(query)  # returns small dict objects internally

print(f"\n🔍 Query: {query}")
print(f"Top {bm25.k} Results (BM25):\n")
for i, r in enumerate(results, 1):
    print(f"[{i}] {r.page_content}")




🔍 Query: library penalty fee rules
Top 3 Results (BM25):

[1] Library rules: penalty fee for late books is $10 per week.
[2] Academic calendar: semester dates, holidays, and exam schedules.
[3] Library timings: Mon-Fri 9AM to 7PM, Sat 10AM to 4PM.


/tmp/ipython-input-796434691.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = bm25.get_relevant_documents(query)  # returns small dict objects internally


In [ ]:


from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.ensemble import EnsembleRetriever
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# --- Your corpus ---
docs_text = [
    "Students must maintain 75% attendance in each course, except CIE. attendance doesnt matter for the CIE course",
    "students must File an FIR at the nearest policestation on losing thier ID cards",
    "Jio works from 02 to 05",
    "Medical exemptions must be submitted within 7 days of return."
]

# --- (1) Dense retriever using your code ---
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_texts(docs_text, embedding=emb, collection_name="college_kb")
dense_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

print("✅ Vector DB Created with", len(docs_text), "documents")

# --- (2) BM25 retriever (lexical) ---
bm25_retriever = BM25Retriever.from_texts(docs_text)
bm25_retriever.k = 2

# --- (3) Hybrid retriever ---
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, dense_retriever],
    weights=[0.4, 0.6]
    k=3
)

# --- (4) Query ---
query = "attendance exemption rules for CIE"
results = hybrid_retriever.get_relevant_documents(query)

print(f"\n🔍 Query: {query}")
print(f"Top {len(results)} results (Hybrid BM25 + Dense):\n")
for i, r in enumerate(results, 1):
    print(f"[{i}] {r.page_content}")


✅ Vector DB Created with 4 documents

🔍 Query: attendance exemption rules for CIE
Top 2 results (Hybrid BM25 + Dense):

[1] Students must maintain 75% attendance in each course, except CIE. attendance doesnt matter for the CIE course
[2] Medical exemptions must be submitted within 7 days of return.


In [ ]:
# rag_query_nebius.py

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "You are a helpful assistant for college policy questions.\n\n"
        "Context:\n{context}\n\n"
        "Question:\n{question}\n\n"
        "Answer clearly using only the context above."
    ),
)

# --- build the RAG chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=hybrid_retriever,
    chain_type_kwargs={"prompt": prompt},
)

# --- query ---
query = "what happens if I miss 75 percent attendance in CIE?"
answer = qa_chain.run(query)

print("\n🔍 Query:", query)
print("\n🧠 Answer:\n", answer)


/tmp/ipython-input-2930952775.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)



🔍 Query: what happens if I miss 75 percent attendance in CIE?

🧠 Answer:
 Since attendance doesn't matter for the CIE course, missing 75% attendance in CIE does not have any consequences.


##TASK


Split into pairs (2 per laptop). Run the provided cells.

For each query in queries (and add one of your own), record:

Top-2 BM25 results (copy exact text)

Top-2 Dense results

Top-3 Hybrid results

For each query, write 1 sentence: which retriever gave the best answer and why (keyword match vs semantic).

Use the Nebius LLM qa.run(...) for one query to produce a final answer; check if the LLM's answer uses correct context.

Submit: a short text file or a single Google Doc with the three queries, the retriever outputs, and your 3 short conclusions.